In [1]:
import qiskit
from qiskit import *
import matplotlib.pyplot as plt
import numpy as np

In [2]:
registers = QuantumRegister(25)
cregisters = ClassicalRegister(25)

circuit = QuantumCircuit(registers, cregisters)

In [3]:
#Row Oracle -- Assigns 4 Qubits into a valid state, such that 3 Qubits are in |0> state and 1 Qubit is in |1> State
#This works with about 70% accuracy

row_oracle = QuantumCircuit(4, name = 'row_oracle')

row_oracle.h(0)
row_oracle.h(3)
row_oracle.x(0)
row_oracle.x(3)
row_oracle.ccx(0,3,1)
row_oracle.x(0)
row_oracle.x(3)
row_oracle.ccx(0,3,2)
row_oracle.cx(2,0)
row_oracle.cx(2,3)

row_oracle.to_gate()
row_oracle.draw()

┌───┐┌───┐     ┌───┐     ┌───┐     
q_0: ┤ H ├┤ X ├──■──┤ X ├──■──┤ X ├─────
     └───┘└───┘┌─┴─┐└───┘  │  └─┬─┘     
q_1: ──────────┤ X ├───────┼────┼───────
               └─┬─┘     ┌─┴─┐  │       
q_2: ────────────┼───────┤ X ├──■────■──
     ┌───┐┌───┐  │  ┌───┐└─┬─┘     ┌─┴─┐
q_3: ┤ H ├┤ X ├──■──┤ X ├──■───────┤ X ├
     └───┘└───┘     └───┘          └───┘

In [4]:
#Append 4 row oracles, to create 4 valid row states

circuit.append(row_oracle, [0,1,2,3])
circuit.append(row_oracle, [4,5,6,7])
circuit.append(row_oracle, [8,9,10,11])
circuit.append(row_oracle, [12,13,14,15])

In [5]:
#Initialize diagonal check ancilla qubits to |1> state.
circuit.x(registers[19])
circuit.x(registers[20])
circuit.x(registers[21])
circuit.x(registers[22])
circuit.x(registers[23])
circuit.x(registers[24])

#AFTER DRAWING, WE SEE A REPEATED PATTERN ON 4 QUBITS, THIS SETS OUR STATE TO A SINGLE 1 per 4 QUBITS
circuit.draw()

┌─────────────┐
 q0_0: ┤0            ├
       │             │
 q0_1: ┤1            ├
       │  row_oracle │
 q0_2: ┤2            ├
       │             │
 q0_3: ┤3            ├
       ├─────────────┤
 q0_4: ┤0            ├
       │             │
 q0_5: ┤1            ├
       │  row_oracle │
 q0_6: ┤2            ├
       │             │
 q0_7: ┤3            ├
       ├─────────────┤
 q0_8: ┤0            ├
       │             │
 q0_9: ┤1            ├
       │  row_oracle │
q0_10: ┤2            ├
       │             │
q0_11: ┤3            ├
       ├─────────────┤
q0_12: ┤0            ├
       │             │
q0_13: ┤1            ├
       │  row_oracle │
q0_14: ┤2            ├
       │             │
q0_15: ┤3            ├
       └─────────────┘
q0_16: ───────────────
                      
q0_17: ───────────────
                      
q0_18: ───────────────
            ┌───┐     
q0_19: ─────┤ X ├─────
            ├───┤     
q0_20: ─────┤ X ├─────
            ├───┤     
q0_21: ─────┤ X ├─────
            ├───┤     
q0_22: ─────┤ X ├─────
            ├───┤     
q0_23: ─────┤ X ├─────
            ├───┤     
q0_24: ─────┤ X ├─────
            └───┘     
c0: 25/═══════════════

In [6]:
# Column check (assuming valid row states)

#Ancilla qubits are initially in a superposition state
circuit.h(16)
circuit.h(17)
circuit.h(18)

pi = 3.1415926536

#This will flip the superposition of the ancilla qubits, assuming a given board qubit is a 1. In other cases, 
#it will remain untouched. In the case of a correct board representation, each ancilla qubit will only be flipped once, 
#and after a second hadamard, will be read in the |1> state.
circuit.cp(pi, 0, 16)
circuit.cp(pi, 4, 16)
circuit.cp(pi, 8, 16)
circuit.cp(pi, 12, 16)
circuit.cp(pi, 1, 17)
circuit.cp(pi, 5, 17)
circuit.cp(pi, 9, 17)
circuit.cp(pi, 13, 17)
circuit.cp(pi, 2, 18)
circuit.cp(pi, 6, 18)
circuit.cp(pi, 10, 18)
circuit.cp(pi, 14, 18)


circuit.h(16)
circuit.h(17)
circuit.h(18)

circuit.draw()

┌─────────────┐                                                        »
 q0_0: ┤0            ├─■──────────────────────────────────────────────────────»
       │             │ │                                                      »
 q0_1: ┤1            ├─┼──────■───────────────────────────────────────────────»
       │  row_oracle │ │      │                                               »
 q0_2: ┤2            ├─┼──────┼──────■────────────────────────────────────────»
       │             │ │      │      │                                        »
 q0_3: ┤3            ├─┼──────┼──────┼────────────────────────────────────────»
       ├─────────────┤ │      │      │                                        »
 q0_4: ┤0            ├─┼──────┼──────┼──────■─────────────────────────────────»
       │             │ │      │      │      │                                 »
 q0_5: ┤1            ├─┼──────┼──────┼──────┼──────■──────────────────────────»
       │  row_oracle │ │      │      │      │      │                          »
 q0_6: ┤2            ├─┼──────┼──────┼──────┼──────┼──────■───────────────────»
       │             │ │      │      │      │      │      │                   »
 q0_7: ┤3            ├─┼──────┼──────┼──────┼──────┼──────┼───────────────────»
       ├─────────────┤ │      │      │      │      │      │                   »
 q0_8: ┤0            ├─┼──────┼──────┼──────┼──────┼──────┼──────■────────────»
       │             │ │      │      │      │      │      │      │            »
 q0_9: ┤1            ├─┼──────┼──────┼──────┼──────┼──────┼──────┼──────■─────»
       │  row_oracle │ │      │      │      │      │      │      │      │     »
q0_10: ┤2            ├─┼──────┼──────┼──────┼──────┼──────┼──────┼──────┼─────»
       │             │ │      │      │      │      │      │      │      │     »
q0_11: ┤3            ├─┼──────┼──────┼──────┼──────┼──────┼──────┼──────┼─────»
       ├─────────────┤ │      │      │      │      │      │      │      │     »
q0_12: ┤0            ├─┼──────┼──────┼──────┼──────┼──────┼──────┼──────┼─────»
       │             │ │      │      │      │      │      │      │      │     »
q0_13: ┤1            ├─┼──────┼──────┼──────┼──────┼──────┼──────┼──────┼─────»
       │  row_oracle │ │      │      │      │      │      │      │      │     »
q0_14: ┤2            ├─┼──────┼──────┼──────┼──────┼──────┼──────┼──────┼─────»
       │             │ │      │      │      │      │      │      │      │     »
q0_15: ┤3            ├─┼──────┼──────┼──────┼──────┼──────┼──────┼──────┼─────»
       └────┬───┬────┘ │P(π)  │      │      │P(π)  │      │      │P(π)  │     »
q0_16: ─────┤ H ├──────■──────┼──────┼──────■──────┼──────┼──────■──────┼─────»
            ├───┤             │P(π)  │             │P(π)  │             │P(π) »
q0_17: ─────┤ H ├─────────────■──────┼─────────────■──────┼─────────────■─────»
            ├───┤                    │P(π)                │P(π)               »
q0_18: ─────┤ H ├────────────────────■────────────────────■───────────────────»
            ├───┤                                                             »
q0_19: ─────┤ X ├─────────────────────────────────────────────────────────────»
            ├───┤                                                             »
q0_20: ─────┤ X ├─────────────────────────────────────────────────────────────»
            ├───┤                                                             »
q0_21: ─────┤ X ├─────────────────────────────────────────────────────────────»
            ├───┤                                                             »
q0_22: ─────┤ X ├─────────────────────────────────────────────────────────────»
            ├───┤                                                             »
q0_23: ─────┤ X ├─────────────────────────────────────────────────────────────»
            ├───┤                                                             »
q0_24: ─────┤ X ├─────────────────────────────────────────────────────────────»
       

In [7]:
# Diagonal check (assuming valid row states)

# Ancilla Qubits are initially in the |1> state

#For rows which are one apart (i.e row 1,2 or row 2,3), 6 comparisons are made, each between qubits requiring diagonal comparison.
#If both in a comparison are in the |1> state, representng a failed diagonal check, the ancilla will be set to |0>

#For rows which are two apart (i.e row 1,3 or row 2,4), 4 comparisons are made, each between qubits requiring diagonal comparison.
#If both in a comparison are in the |1> state, representng a failed diagonal check, the ancilla will be set to |0>

#For rows which are three apart (i.e row 1,4), 2 comparisons are made, each between qubits requiring diagonal comparison.
#If both in a comparison are in the |1> state, representng a failed diagonal check, the ancilla will be set to |0>

#If all 6 ancilla qubits are ultimately measured in the |1> state, the diagonal condition is passed. 

#Compare Row 1,2
circuit.ccx(registers[0], registers[5], registers[19])
circuit.ccx(registers[1], registers[4], registers[19])
circuit.ccx(registers[1], registers[6], registers[19])
circuit.ccx(registers[2], registers[5], registers[19])
circuit.ccx(registers[2], registers[7], registers[19])
circuit.ccx(registers[3], registers[6], registers[19])

#Compare Row 1,3
circuit.ccx(registers[0], registers[10], registers[20])
circuit.ccx(registers[1], registers[11], registers[20])
circuit.ccx(registers[2], registers[8], registers[20])
circuit.ccx(registers[3], registers[9], registers[20])

#Compare Row 1,4
circuit.ccx(registers[0], registers[15], registers[21])
circuit.ccx(registers[3], registers[12], registers[21])

#Compare Row 2,3
circuit.ccx(registers[4], registers[9], registers[22])
circuit.ccx(registers[5], registers[8], registers[22])
circuit.ccx(registers[5], registers[10], registers[22])
circuit.ccx(registers[6], registers[9], registers[22])
circuit.ccx(registers[6], registers[11], registers[22])
circuit.ccx(registers[7], registers[10], registers[22])

#Compare Row 2,4
circuit.ccx(registers[4], registers[14], registers[23])
circuit.ccx(registers[5], registers[15], registers[23])
circuit.ccx(registers[6], registers[12], registers[23])
circuit.ccx(registers[7], registers[13], registers[23])

#Compare Row 3,4
circuit.ccx(registers[8], registers[13], registers[24])
circuit.ccx(registers[9], registers[12], registers[24])
circuit.ccx(registers[9], registers[14], registers[24])
circuit.ccx(registers[10], registers[13], registers[24])
circuit.ccx(registers[10], registers[15], registers[24])
circuit.ccx(registers[11], registers[14], registers[24])



circuit.measure(registers, cregisters)
circuit.draw()


┌─────────────┐                                                        »
 q0_0: ┤0            ├─■──────────────────────────────────────────────────────»
       │             │ │                                                      »
 q0_1: ┤1            ├─┼──────■───────────────────────────────────────────────»
       │  row_oracle │ │      │                                               »
 q0_2: ┤2            ├─┼──────┼──────■────────────────────────────────────────»
       │             │ │      │      │                                        »
 q0_3: ┤3            ├─┼──────┼──────┼────────────────────────────────────────»
       ├─────────────┤ │      │      │                                        »
 q0_4: ┤0            ├─┼──────┼──────┼──────■─────────────────────────────────»
       │             │ │      │      │      │                                 »
 q0_5: ┤1            ├─┼──────┼──────┼──────┼──────■──────────────────────────»
       │  row_oracle │ │      │      │      │      │                          »
 q0_6: ┤2            ├─┼──────┼──────┼──────┼──────┼──────■───────────────────»
       │             │ │      │      │      │      │      │                   »
 q0_7: ┤3            ├─┼──────┼──────┼──────┼──────┼──────┼───────────────────»
       ├─────────────┤ │      │      │      │      │      │                   »
 q0_8: ┤0            ├─┼──────┼──────┼──────┼──────┼──────┼──────■────────────»
       │             │ │      │      │      │      │      │      │            »
 q0_9: ┤1            ├─┼──────┼──────┼──────┼──────┼──────┼──────┼──────■─────»
       │  row_oracle │ │      │      │      │      │      │      │      │     »
q0_10: ┤2            ├─┼──────┼──────┼──────┼──────┼──────┼──────┼──────┼─────»
       │             │ │      │      │      │      │      │      │      │     »
q0_11: ┤3            ├─┼──────┼──────┼──────┼──────┼──────┼──────┼──────┼─────»
       ├─────────────┤ │      │      │      │      │      │      │      │     »
q0_12: ┤0            ├─┼──────┼──────┼──────┼──────┼──────┼──────┼──────┼─────»
       │             │ │      │      │      │      │      │      │      │     »
q0_13: ┤1            ├─┼──────┼──────┼──────┼──────┼──────┼──────┼──────┼─────»
       │  row_oracle │ │      │      │      │      │      │      │      │     »
q0_14: ┤2            ├─┼──────┼──────┼──────┼──────┼──────┼──────┼──────┼─────»
       │             │ │      │      │      │      │      │      │      │     »
q0_15: ┤3            ├─┼──────┼──────┼──────┼──────┼──────┼──────┼──────┼─────»
       └────┬───┬────┘ │P(π)  │      │      │P(π)  │      │      │P(π)  │     »
q0_16: ─────┤ H ├──────■──────┼──────┼──────■──────┼──────┼──────■──────┼─────»
            ├───┤             │P(π)  │             │P(π)  │             │P(π) »
q0_17: ─────┤ H ├─────────────■──────┼─────────────■──────┼─────────────■─────»
            ├───┤                    │P(π)                │P(π)               »
q0_18: ─────┤ H ├────────────────────■────────────────────■───────────────────»
            ├───┤                                                             »
q0_19: ─────┤ X ├─────────────────────────────────────────────────────────────»
            ├───┤                                                             »
q0_20: ─────┤ X ├─────────────────────────────────────────────────────────────»
            ├───┤                                                             »
q0_21: ─────┤ X ├─────────────────────────────────────────────────────────────»
            ├───┤                                                             »
q0_22: ─────┤ X ├─────────────────────────────────────────────────────────────»
            ├───┤                                                             »
q0_23: ─────┤ X ├─────────────────────────────────────────────────────────────»
            ├───┤                                                             »
q0_24: ─────┤ X ├─────────────────────────────────────────────────────────────»
       

In [8]:
from qiskit_ibm_runtime import QiskitRuntimeService
import pydantic

service = QiskitRuntimeService(
    channel='ibm_quantum',
    token='34c7d3eed0575b6cd66edeb220d76ea1b6bd87a0f739d012e3027f8a26325419c4759e3af104b307f18f9142174f5501dc404cf33e4f1e7d28a6681902d8ea9b'
)

backend = service.least_busy(simulator=False, operational=True)




In [ ]:
transpiled_circuit = transpile(circuit, backend)
job = backend.run(transpiled_circuit, shots=1024)
results = job.result()
answer = results.get_counts()

In [ ]:
answer